In [ ]:
pip install googlesearch-python


In [1]:
from datetime import datetime
import speech_recognition as sr
import pyttsx3
import webbrowser
import wikipedia
import wolframalpha
import openai
import os
from dotenv import load_dotenv
import googlesearch
import requests
import pywhatkit

# Load environment variables from .env file
load_dotenv()

# Mute ALSA errors
from ctypes import *
from contextlib import contextmanager

ERROR_HANDLER_FUNC = CFUNCTYPE(None, c_char_p, c_int, c_char_p, c_int, c_char_p)

def py_error_handler(filename, line, function, err, fmt):
    pass

c_error_handler = ERROR_HANDLER_FUNC(py_error_handler)

@contextmanager
def noalsaerr():
    try:
        asound = cdll.LoadLibrary('libasound.so')
        asound.snd_lib_error_set_handler(c_error_handler)
        yield
        asound.snd_lib_error_set_handler(None)
    except:
        yield
        print('')

### PARAMETERS ###
tts_type = 'local'  # google or local

r = sr.Recognizer()

engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
# Change the voice to sound more like JARVIS
jarvis_voice_id = voices[0].id
engine.setProperty('voice', jarvis_voice_id)

def speak(text, rate=120):
    print(f"Speaking: {text}")  # Debug print
    try:
        if tts_type == 'local':
            engine.setProperty('rate', rate)
            engine.say(text)
            engine.runAndWait()
    except KeyboardInterrupt:
        return

def take_command():
    with sr.Microphone() as source:
        print("Listening.....")
        r.pause_threshold = 1
        audio = r.listen(source)
    try:
        print("Wait for a few moments")
        query = r.recognize_google(audio, language="en-in")
        print("User said ", query)
        return query.lower()
    except Exception as e:
        print(e)
        speak("Say that again, please")
        return None

def wish_me():
    hour = int(datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")

chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
webbrowser.register('chrome', None, webbrowser.BackgroundBrowser(chrome_path))

appId = '5R49J7-J888YX9J2V'
wolframClient = wolframalpha.Client(appId)

def parse_command():
    with noalsaerr():
        listener = sr.Recognizer()
        print('Listening for a command')

        with sr.Microphone() as source:
            listener.pause_threshold = 2
            input_speech = listener.listen(source)

        try:
            print('Recognizing speech...')
            query = listener.recognize_google(input_speech, language='en_gb')
            print(f'The input speech was: {query}')
            return query.lower()
        except Exception as exception:
            print('I did not quite catch that')
            print(exception)
            return 'None'

def search_wikipedia(keyword=''):
    try:
        search_results = wikipedia.search(keyword)
        if not search_results:
            return 'No results found on Wikipedia.'

        for result in search_results[:3]:
            try:
                wiki_page = wikipedia.page(result)
                return wiki_page.summary
            except wikipedia.DisambiguationError:
                continue
            except wikipedia.PageError:
                continue

        return 'No valid page found on Wikipedia.'

    except Exception as e:
        print(f"An error occurred while searching Wikipedia: {e}")
        return 'An error occurred while searching Wikipedia.'

def list_or_dict(var):
    if isinstance(var, list):
        return var[0]['plaintext']
    else:
        return var['plaintext']

def search_wolframalpha(keyword=''):
    try:
        response = wolframClient.query(keyword)
        if response['@success'] == 'false':
            return 'I could not compute that.'
        else:
            result = ''
            pod0 = response['pod'][0]
            pod1 = response['pod'][1]
            if (('result') in pod1['@title'].lower()) or (pod1.get('@primary', 'false') == 'true') or ('definition' in pod1['@title'].lower()):
                result = list_or_dict(pod1['subpod'])
                return result.split('(')[0]
            else:
                question = list_or_dict(pod0['subpod'])
                question = question.split('(')[0]
                return question
    except Exception as e:
        print(f"An error occurred with WolframAlpha: {e}")
        return 'Unable to compute'

def query_openai(prompt=""):
    openai.organization = os.getenv('OPENAI_ORG')
    openai.api_key = os.getenv('OPENAI_API_KEY')
    try:
        response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, temperature=0.3, max_tokens=80)
        print(f"OpenAI response: {response}")  # Debug print
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error querying OpenAI: {e}")  # Debug print
        return "Sorry, I could not process that request."

def open_website(url):
    url_mapping = {
        "google maps": "https://maps.google.com",
        "youtube": "https://www.youtube.com",
        "google": "https://www.google.com",
        "facebook": "https://www.facebook.com",
    }

    if url in url_mapping:
        full_url = url_mapping[url]
    else:
        if not url.startswith('http'):
            url = 'https://' + url + '.com'
        full_url = url

    try:
        webbrowser.open(full_url)
        speak(f"Opening {url}")
    except Exception as e:
        print(e)
        speak("Sorry, I am unable to open the website.")

def open_google_maps(location):
    base_url = "https://www.google.com/maps/dir/?api=1&destination="
    destination = location.replace(" ", "+")
    full_url = base_url + destination
    try:
        webbrowser.open(full_url)
        speak(f"Navigating to {location}")
    except Exception as e:
        print(e)
        speak("Sorry, I am unable to navigate to the location.")

def get_time():
    now = datetime.now().strftime("%H:%M:%S")
    speak(f"The current time is {now}")

def get_date():
    today = datetime.now().strftime("%Y-%m-%d")
    speak(f"Today's date is {today}")

def get_weather(city):
    api_key = os.getenv('WEATHER_API_KEY')
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city
    response = requests.get(complete_url)
    data = response.json()
    if data["cod"] != "404":
        main = data["main"]
        temperature = main["temp"]
        pressure = main["pressure"]
        humidity = main["humidity"]
        weather_desc = data["weather"][0]["description"]
        temp_celsius = temperature - 273.15
        speak(f"The temperature in {city} is {temp_celsius:.2f} degrees Celsius with {weather_desc}. The atmospheric pressure is {pressure} hPa and humidity is {humidity}%.")
    else:
        speak("City not found")

def search_google(query):
    search_url = f"https://www.google.com/search?q={query.replace(' ', '+')}"
    try:
        webbrowser.open(search_url)
        speak(f"Searching Google for {query}")
    except Exception as e:
        print(e)
        speak("Sorry, I am unable to perform the search.")

def play_youtube_video(query):
    pywhatkit.playonyt(query)
    speak(f"Playing {query} on YouTube")

def handle_query(query):
    if 'wikipedia' in query:
        speak('Searching Wikipedia...')
        query = query.replace('wikipedia', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'search wikipedia for' in query:
        speak('Searching Wikipedia...')
        query = query.replace('search wikipedia for', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'tell me about' in query:
        speak('Searching Wikipedia...')
        query = query.replace('tell me about', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'what is' in query:
        speak('Searching Wikipedia...')
        query = query.replace('what is', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'who is' in query:
        speak('Searching Wikipedia...')
        query = query.replace('who is', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'give me detailed information about' in query:
        speak('Searching Wikipedia...')
        query = query.replace('give me detailed information about', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'define' in query:
        speak('Searching Wikipedia...')
        query = query.replace('define', '').strip()
        results = search_wikipedia(query)
        speak(results)
    elif 'open youtube' in query:
        open_website('youtube')
    elif 'open google' in query:
        open_website('google')
    elif 'open facebook' in query:
        open_website('facebook')
    elif 'time' in query:
        get_time()
    elif 'date' in query:
        get_date()
    elif 'weather' in query:
        speak("Which city's weather would you like to know?")
        city = take_command()
        get_weather(city)
    elif 'calculate' in query or '+' in query or '-' in query or '*' in query or '/' in query:
        try:
            result = eval(query)
            speak(f"The result is {result}")
        except Exception as e:
            speak("I could not calculate that. Please try again.")
    elif 'ask' in query:
        speak('What would you like to ask?')
        question = take_command()
        answer = search_wolframalpha(question)
        speak(answer)
    elif 'openai' in query:
        speak('What is your prompt for OpenAI?')
        prompt = take_command()
        answer = query_openai(prompt)
        speak(answer)
    elif 'google search' in query or 'search google for' in query or 'search' in query:
        query = query.replace('google search', '').replace('search google for', '').replace('search', '').strip()
        search_google(query)
    elif 'sreenidhi institute of science and technology' in query:
        speak('Searching information about Sreenidhi Institute of Science and Technology...')
        results = search_wikipedia('Sreenidhi Institute of Science and Technology')
        speak(results)
    elif 'play' in query and 'youtube' in query:
        query = query.replace('play', '').replace('on youtube', '').strip()
        play_youtube_video(query)
    elif 'navigate to' in query:
        location = query.replace('navigate to', '').strip()
        open_google_maps(location)
    elif 'exit' in query or 'quit' in query or 'stop' in query:
        speak("Goodbye!")
        return False
    else:
        speak("I did not understand that. Please try again.")
    
    return True

if __name__ == "__main__":
    wish_me()
    while True:
        query = take_command()
        if query is None:
            continue

        if not handle_query(query.lower()):
            break


Speaking: Good Morning!
Listening.....
Wait for a few moments
User said  navigate to Gachibowli
Speaking: Navigating to gachibowli
Listening.....
Wait for a few moments

Speaking: Say that again, please
Listening.....
Wait for a few moments
User said  is it really today
Speaking: I did not understand that. Please try again.
Listening.....
Wait for a few moments

Speaking: Say that again, please
Listening.....
Wait for a few moments

Speaking: Say that again, please
Listening.....


OSError: [Errno -9988] Stream closed